# Assignment #1: Performance Benchmarking of Cryptographic Mechanisms



Trabalho por: 
- David Rodrigues up202303949
- Diana Pereira up202304476
- Simão Gomes up202304752

### Index
[0. Libreries and Modules](#importing-required-libraries-and-modules) <br>
[A. Generating text files](#a-generating-random-text-files) <br>
[B. AES](#b-encrypting-and-decrypting-using-aes)<br>
- [B.1. Results](#b1-comparing-aes-results)<br>
- [B.2. Padding](#b2-padding-vs-no-padding)<br>
[C. RSA]

### Importing required libraries and modules

In [1]:
import os
import timeit

from cryptography.hazmat.primitives import hashes, padding
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

### A. Generating random text files

Files with the following sizes:
- Advanced Encription Standard (in bytes): 8, 64, 512, 4096, 32768, 262144, 2097152
- Rivest-Shamir-Adleman (in bytes): 8, 64, 512, 4096, 32768, 262144, 2097152
- Secure Hash Algorithm (in bytes): 2, 4, 8, 16, 32, 64, 128

In [4]:
tamanhosAES = [8,64,512,4096,32768,262144,2097152]
tamanhosRSA = [8,64,512,4096,32768,262144,2097152]
tamanhosSHA = [2,4,8,16,32,64,128]

os.makedirs("Ficheiros",exist_ok=True) #exist_ok para resolver o erro de já existir o diretório "FileExistsError"

def gerar_ficheiros(tamanho,nome):
    with open(f"Ficheiros/{nome}", "wb") as file:
        file.write(os.urandom(tamanho))

for t in tamanhosAES:
    gerar_ficheiros(t, f"AES_{t}.txt")

for t in tamanhosRSA:
    gerar_ficheiros(t, f"RSA_{t}.txt")

for t in tamanhosSHA:
    gerar_ficheiros(t, f"SHA_{t}.txt")

### B. Encrypting and decrypting using AES
To get statistically significant results we will repeat the encription->decription procedure several times for each file 
We will be using padding  

In [ ]:
key_AES=os.urandom(32) #key de 256 bits
iv_AES=os.urandom(16) #iv de 128 bits

def encrypt_AES(original):
    cipher = Cipher(algorithms.AES(key_AES), modes.CBC(iv_AES))
    encryptor = cipher.encryptor()

    #padding para tornar o texto múltiplo de 16 bytes
    padder = padding.PKCS7(128).padder()
    padded = padder.update(original) + padder.finalize()

    return encryptor.update(padded) + encryptor.finalize()

def decrypt_AES(encrypted):
    cipher = Cipher(algorithms.AES(key_AES), modes.CBC(iv_AES))
    decryptor = cipher.decryptor()

    decrypted= decryptor.update(encrypted) + decryptor.finalize()

    unpadder = padding.PKCS7(128).unpadder()

    return unpadder.update(decrypted) + unpadder.finalize()

resultados_AES = []
reps = 100

for t in tamanhosAES:
    with open(f"Ficheiros/AES_{t}.txt","rb") as file:
        original = file.read()

    tempo_enc = 0
    tempo_dec = 0
    
    for i in range(reps):
        start = timeit.default_timer()
        encrypted = encrypt_AES(original)
        end = timeit.default_timer()
        tempo_enc += end - start

        start = timeit.default_timer()
        decrypt_AES(encrypted)
        end = timeit.default_timer()
        tempo_dec += end - start

    tempo_enc /= reps
    tempo_dec /= reps

    tempo_enc *= 1e6 #converte para microsegundos
    tempo_dec *= 1e6

    resultados_AES.append((t, tempo_enc, tempo_dec))

    print(t , "bytes: encryption time: ", tempo_enc, "micros, decryption time: ", tempo_dec, "micros")


8 bytes: encryption time:  86.54800039948896 micros, decryption time:  83.95799872232601 micros
64 bytes: encryption time:  41.098998044617474 micros, decryption time:  40.800999267958105 micros
512 bytes: encryption time:  40.63500004122034 micros, decryption time:  39.61299953516573 micros
4096 bytes: encryption time:  49.116000009234995 micros, decryption time:  45.4130006255582 micros
32768 bytes: encryption time:  114.64500101283193 micros, decryption time:  83.27199990162626 micros
262144 bytes: encryption time:  365.3510008007288 micros, decryption time:  162.76700131129473 micros
2097152 bytes: encryption time:  5284.178002038971 micros, decryption time:  3797.110998420976 micros


#### B.1. Comparing AES Results

#### B.2. Padding vs No Padding

### C. Encrypting and decrypting with RSA